In [1]:
import numpy as np
import gymnasium
from run.train import configuration
import panda_gym
import pybullet
from sb3_contrib import TQC
import evaluate

C:\Users\ladmin\PycharmProjects\panda-gym\venv\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment PandaReachAO-v3 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [2]:
# standard settings
reward_type="sparse"
n_substeps=20
max_ep_steps = 200
goal_condition="reach"
task_observations={'obstacles': "vectors+past", 'prior': None}
num_episodes = 20
#register envs
panda_gym.register_reach_ao(max_ep_steps)

# visual stuff
sphere_list = []
sphere_count = 0
human = True # render the environment during evaluation

In [3]:
# training env
evaluation_scenario="wangexp_3"
prior_orientation="fkine"
show_goal_space = True
cameraTargetPosition = (0.06289377063512802, 0.05091303586959839, 0.27599984407424927)
cameraDistance = 1.359999656677246
cameraPitch = -33.20000076293945
cameraYaw = 51.800025939941406

# narrow_tunnel
evaluation_scenario="narrow_tunnel"
prior_orientation="left"
show_goal_space = True
cameraTargetPosition = (0.6126745939254761, -0.002699121832847595, 0.3287460505962372)
cameraDistance = 1.119999885559082
cameraPitch = -42.800045013427734
cameraYaw = 221.39991760253906

# library 2
evaluation_scenario="library2"
prior_orientation="back"
show_goal_space = True
cameraTargetPosition = (0.09789270907640457, 0.09457920491695404, 0.3600000739097595)
cameraDistance = 1.5399994850158691
cameraPitch = -36.4000129699707
cameraYaw = 49.800018310546875

# workshop
evaluation_scenario="workshop"
prior_orientation="fkine"
show_goal_space = True
cameraTargetPosition = (-0.05694245547056198, -0.08760415762662888, 0.3400000035762787)
cameraDistance = 1.299999713897705
cameraPitch = -42.8000373840332
cameraYaw = 224.19996643066406

# wall
evaluation_scenario="wall"
prior_orientation="fkine"
show_goal_space = True
cameraTargetPosition = (0.03229629248380661, 0.10305798053741455, 0.012000000104308128)
cameraDistance = 1.5399994850158691
cameraPitch = -30.399999618530273
cameraYaw = 66.60003662109375

In [4]:
# get env
work_env = gymnasium.make(configuration["env_name"], render=False, control_type="js",
                     obs_type=configuration["obs_type"], goal_distance_threshold=0.05,
                     goal_condition=goal_condition,
                     reward_type=reward_type, limiter=configuration["limiter"],
                     show_goal_space=False, scenario=evaluation_scenario,
                     randomize_robot_pose=False,  # if evaluation_scenario != "wang_3" else True,
                     task_observations=task_observations,
                     truncate_on_collision=True,
                     terminate_on_success=True, 
                     show_debug_labels=False, n_substeps=n_substeps)


In [5]:
# get rendered env
env = gymnasium.make(configuration["env_name"],
                     render=True, control_type="js",
                     obs_type=configuration["obs_type"], goal_distance_threshold=0.05,
                     goal_condition=goal_condition,
                     reward_type=reward_type, limiter=configuration["limiter"],
                     show_goal_space=show_goal_space, scenario=evaluation_scenario,
                     randomize_robot_pose=False,  # if evaluation_scenario != "wang_3" else True,
                     task_observations=task_observations,
                     truncate_on_collision=True,
                     terminate_on_success=True,
                     show_debug_labels=False, n_substeps=n_substeps)

pybullet.resetDebugVisualizerCamera(cameraDistance=cameraDistance, cameraYaw=cameraYaw, cameraPitch=cameraPitch, cameraTargetPosition=cameraTargetPosition, physicsClientId=1)



In [10]:
# collect prior results
work_env.robot.control_type = "jsd"
results, metrics = evaluate.evaluate_ensemble(["prior"], work_env, human=False, num_episodes=num_episodes, deterministic=True,
                                     strategy="",
                                     scenario_name=evaluation_scenario,
                                     prior_orientation=prior_orientation)
ee_pos = metrics["end_effector_positions"]
path_success = metrics["path_success"]
done_events  = metrics["done_events"]

wangexp_3: 100%|██████████| 20/20 [00:26<00:00,  1.33s/it]


In [ ]:
# collect agent results
work_env.robot.control_type = "js"
model_names = ["logical-cherry-949"]# evaluate.trained_models['mt_cl']
ensemble = []
for model_name in model_names:
    ensemble.append(TQC.load(fr"../run/run_data/wandb/{model_name}/files/best_model.zip", env=work_env,
                     custom_objects={"action_space": gymnasium.spaces.Box(-1.0, 1.0, shape=(7,),dtype=np.float32)}))  # for some reason it won't read action space sometimes)

results, metrics = evaluate.evaluate_ensemble(ensemble, work_env, human=human, num_episodes=num_episodes, deterministic=True,
                                     strategy="variance_only",
                                     scenario_name=evaluation_scenario,
                                     prior_orientation="")
ee_pos = metrics["end_effector_positions"]
path_success = metrics["path_success"]
done_events  = metrics["done_events"]
work_env.close()

pybullet.setRealTimeSimulation(0)
loggingID = pybullet.startStateLogging(loggingType=pybullet.STATE_LOGGING_VIDEO_MP4, fileName = f"videos/{evaluation_scenario}_mt-cl-ensemble.mp4", physicsClientId=1)

In [11]:
# visualize/play prior results
env.robot.control_type = "jsd"
#pybullet.setRealTimeSimulation(enableRealTimeSimulation=1, physicsClientId=1)

_, _ = evaluate.evaluate_ensemble(["prior"], env, human=True, num_episodes=num_episodes, deterministic=True,
                                     strategy="",
                                     scenario_name=evaluation_scenario,
                                     prior_orientation="",
                                  pre_calc_metrics=metrics)


wangexp_3: 100%|██████████| 20/20 [01:06<00:00,  3.34s/it]


pybullet.stopStateLogging(loggingID)
env.close()

env.close()

work_env.robot.control_type = "js"
model_name = "firm-pond-79"
model = TQC.load(fr"../run/run_data/wandb/{model_name}/files/best_model.zip", env=work_env,
                 custom_objects={"action_space": gymnasium.spaces.Box(-1.0, 1.0, shape=(7,),
                                                                      dtype=np.float32)})  # for some reason it won't read action space sometimes

results, metrics = evaluate.evaluate_ensemble([model], work_env, human=human, num_episodes=100, deterministic=True,
                                     strategy="variance_only",
                                     scenario_name=evaluation_scenario,
                                     prior_orientation="")
ee_pos = metrics["end_effector_positions"]
path_success = metrics["path_success"]
done_events  = metrics["done_events"]

# get rendered env
env = gymnasium.make(configuration["env_name"],
                     render=True, control_type="js",
                     obs_type=configuration["obs_type"], goal_distance_threshold=0.05,
                     goal_condition="halt",
                     reward_type=reward_type, limiter=configuration["limiter"],
                     show_goal_space=False, scenario=evaluation_scenario,
                     randomize_robot_pose=False,  # if evaluation_scenario != "wang_3" else True,
                     task_observations="vectors+all",
                     truncate_on_collision=True,
                     terminate_on_success=True,
                     show_debug_labels=True, n_substeps=n_substeps)

In [ ]:

# pybullet.changeVisualShape(objectUniqueId=0, linkIndex=1, rgbaColor=[1,1,1,0.5], physicsClientId=1)
# pybullet.changeVisualShape(objectUniqueId=0, linkIndex=3, rgbaColor=[1,1,1,1], physicsClientId=1)
# pybullet.changeVisualShape(objectUniqueId=0, linkIndex=2, rgbaColor=[1,1,1,0.5])
# pybullet.changeVisualShape(objectUniqueId=0, linkIndex=1, rgbaColor=[1,1,1,0.5])
# pybullet.changeVisualShape(objectUniqueId=0, linkIndex=0, rgbaColor=[1,1,1,0.5])

# todo: draw a circle
time.sleep(3)
radius = 0.9
theta = 0.49*np.pi
phi = 2*np.pi


def get_next_point(radius, phi, theta):
    return [radius * np.sin(theta) * np.cos(phi), radius * np.sin(theta) * np.sin(phi), radius * np.cos(theta)]

inc = -phi/4
xyz1 = get_next_point(radius,inc,theta)
th= 0.0
while th < theta:
    while inc < phi/4:
        inc += 0.01
        xyz2 = get_next_point(radius, inc, theta)
        pybullet.addUserDebugLine(lineFromXYZ=xyz1, lineToXYZ=xyz2, lineColorRGB=np.array([0,1,0]), physicsClientId=1, lifeTime=0, lineWidth=2)
        time.sleep(0.01)
        xyz1 = xyz2
    th += 0.01

# inc = -0.5*np.pi
# xyz1=get_next_point(phi, theta*inc)
# while inc < 0.5*np.pi:
#     inc += 0.01
#     xyz2 = get_next_point(phi, theta*inc)
#     pybullet.addUserDebugLine(lineFromXYZ=xyz1, lineToXYZ=xyz2, lineColorRGB=np.array([0,1,0]), physicsClientId=1, lifeTime=0, lineWidth=2)
#     time.sleep(0.01)
#     xyz1 = xyz2

inc = -0.5*np.pi
xyz1=get_next_point(radius, phi/2, theta*inc)
while inc < 0.5*np.pi:
    inc += 0.01
    xyz2 = get_next_point(radius, phi/4, theta*inc)
    pybullet.addUserDebugLine(lineFromXYZ=xyz1, lineToXYZ=xyz2, lineColorRGB=np.array([0,1,0]), physicsClientId=1, lifeTime=0, lineWidth=2)
    time.sleep(0.01)
    xyz1 = xyz2

rad = -0.9
xyz1=[0.0, rad, 0.01]
while rad < radius:
    rad += 0.01
    xyz2 = [0.0, rad, 0.01]
    pybullet.addUserDebugLine(lineFromXYZ=xyz1, lineToXYZ=xyz2, lineColorRGB=np.array([0,1,0]), physicsClientId=1, lifeTime=0, lineWidth=2)
    time.sleep(0.01)
    xyz1 = xyz2


time.sleep(3)
env.task.create_goal_outline()
while True:
    time.sleep(1)
    env.reset()

In [ ]:
#env.close()